In [1]:
import os
import pandas as pd
from itertools import islice

### 1 - Load and parse helper files for loading raw OPD data file

In [5]:
data_path = os.path.join('..', 'data', 'raw')

#helpers = []
#for file in os.listdir(data_path):
#    if file.endswith('.sps'):
#        helpers.append(pd.read_csv(os.path.join(data_path, file)))

In [7]:
with open(os.path.join(data_path, 'opd2011.sps'), 'r', encoding='windows-1252') as f:
    # Keep the trunk after 'DATA LIST...' and before the first '.'
    for i, line in enumerate(f):
        if line.startswith('DATA LIST'):
            start = i + 1
        if line.startswith('.'):
            end = i
            break

    #helper = [line.strip() for line in helper[start:end]]
    f.seek(0)
    helper = list(islice(f, start, end))
    helper = [line.strip() for line in helper]

# Convert the helper to a DataFrame, split each line by whitespace
helper = pd.DataFrame([line.replace(' (A)', '').split(maxsplit=1) for line in helper]).rename(columns={0:'name'})

# Remove the rows that ends with digits followed by 'D'
helper = helper[~helper['name'].str.contains(r'\dD$')]

In [8]:
helper['width'] = helper[1].apply(lambda x: abs(eval(x)) + 1 if ('-' in x) else 1)

In [19]:
opd = pd.read_fwf(os.path.join(data_path, 'opd2011'), widths=helper['width'], names=helper['name'], header=None)
opd

,VMONTH,VDAYR,AGE,SEX,ETHUN,RACEUN,PAYPRIV,PAYMCARE,PAYMCAID,PAYWKCMP,...,TGS,TGSRES,DAYDTGS,FBG,FBGRES,DAYDFBG,INJR1,INJR2,INJDETR1,INJDETR2
0,1,2,63,2,2,1,1,0,0,0,...,2,-9,-900,1,91,-800,0,0,5,5
1,1,2,72,1,2,1,0,1,1,0,...,1,230,44,1,145,-210,0,0,5,5
2,1,3,77,1,2,1,0,1,1,0,...,1,128,-84,1,98,-5,0,0,5,5
3,1,3,27,2,2,1,0,0,0,0,...,1,494,-15,1,102,-15,0,0,5,5
4,1,4,67,2,2,1,1,1,0,0,...,2,-9,-900,1,127,22,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32228,10,6,71,1,-9,-9,1,1,0,0,...,-9,-9,-900,-9,-9,-900,0,0,5,5
32229,10,6,59,1,1,-9,0,1,1,0,...,-9,-9,-900,-9,-9,-900,0,0,5,5
32230,10,6,54,2,-9,1,1,1,0,0,...,-9,-9,-900,-9,-9,-900,0,0,5,5
32231,10,6,71,2,-9,1,1,1,0,0,...,-9,-9,-900,-9,-9,-900,0,0,5,5


In [22]:
opd.iloc[:, 20:40]

,RFV3,PRIMCARE,REFER,SENBEFOR,PASTVIS,MAJOR,DIAG1,DIAG2,DIAG3,PRDIAG1,PRDIAG2,PRDIAG3,ARTHRTIS,ASTHMA,CANCER,CEBVD,CRF,CHF,COPD,DEPRN
0,24200,1,-7,1,3,1,37230,7291-,5641-,0,0,0,1,0,0,0,0,0,0,0
1,-9,1,-7,1,4,2,25000,2724-,42731,0,0,0,1,0,0,1,0,0,0,1
2,25100,1,-7,1,19,1,4019-,42731,V5861,0,0,0,0,0,0,0,0,1,1,0
3,-9,1,-7,1,30,2,340--,7061-,29690,0,0,0,0,0,0,0,0,0,0,1
4,-9,1,-7,1,3,2,2113-,V1272,4414-,0,0,0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32228,-9,2,-8,2,-7,1,5533-,5303-,-9,0,0,-7,1,0,1,0,0,0,0,0
32229,-9,2,1,2,-7,1,2113-,4550-,-9,0,0,-7,0,0,1,0,0,0,1,0
32230,-9,2,-8,2,-7,1,5533-,53550,4550-,0,0,0,0,0,0,0,0,0,0,0
32231,-9,2,1,2,-7,5,2113-,56210,4550-,0,0,0,0,0,0,0,0,0,0,0


In [21]:
opd.PROC1R.value_counts()

PROC1R
-9       20261
 8970     2115
 8939     1946
 9509      492
 8917      435
         ...  
 7124        1
 6720        1
 6563        1
 5440        1
 4292        1
Name: count, Length: 596, dtype: int64